In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [3]:
df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


Пустые значения

In [ ]:
df.dropna()

In [ ]:
df[df['user_id'].isna() == True]

In [ ]:
df['event_type'].value_counts(normalize=True)

---

In [ ]:
date = df['event_date'].unique()
print('Данные с {} по {}'.format(date[0], date[-1]))

---

Взаимодействие по дням

In [ ]:
tmp = df.groupby(['event_date'], as_index=False) \
    .agg({'event_timestamp': 'count'}) \
    .sort_values(['event_date'], ascending=True) \
    .rename(columns={'event_timestamp': 'count'})

In [ ]:
tmp

In [ ]:
fig = px.bar(
    data_frame=tmp,
    x='event_date',
    y='count',
    title='All interactions'
)
fig.show()

In [ ]:
tmp = df.query('event_type != "show_vacancy"').groupby(['event_date'], as_index=False) \
    .agg({'event_timestamp': 'count'}) \
    .sort_values(['event_date'], ascending=True) \
    .rename(columns={'event_timestamp': 'count'})

fig = px.bar(
    data_frame=tmp,
    x='event_date',
    y='count',
    title='Without show events, only clicks'
)
fig.show()

Просадки по выходным

---

In [ ]:
vac = df['vacancy_id_'].value_counts()

In [ ]:
vac

In [ ]:
print('Всего вакансий: ', len(vac))

CTR

In [ ]:
vacancy_rel = df.pivot_table(index='vacancy_id_', columns='event_type', values='event_timestamp', aggfunc='count', fill_value=0).reset_index()
vacancy_rel['ctr'] = vacancy_rel.apply(lambda x: (x['preview_click_vacancy'] + x['click_response'] + x['click_contacts'] + x['preview_click_response']+ x['click_favorite'] + x['preview_click_favorite'] + x['preview_click_contacts'] + x['click_phone'] + x['preview_click_phone'])/x['show_vacancy'] if x['show_vacancy'] != 0 else 0, axis=1)

In [ ]:
vacancy_rel['ctr'].sort_values(ascending=False)

---

In [ ]:
user = df['user_id'].value_counts()

In [ ]:
user

In [ ]:
print('Всего юзеров: ', len(user))

Взаимодействующие юзеры

In [ ]:
active_users = df.query('event_type != "show_vacancy"')['user_id'].value_counts()

In [ ]:
print('Кол-во взаимодействующих юзеров: ', len(active_users))

---

Кол-во сессий

In [ ]:
tmp_cookie = df.groupby(['user_id'], as_index=False)['cookie_id'].nunique() \
    .sort_values(['cookie_id'], ascending=False) \
    .reset_index() \
    .rename(columns={'cookie_id': 'count'})

In [ ]:
tmp_cookie

In [ ]:
tmp_cookie['count'].mean()

---